In [1]:
import keras
from keras.layers import Activation, Dense, Dropout, Conv2D, \
                         Flatten, MaxPooling2D
from keras.models import Sequential
import librosa
import librosa.display
import numpy as np
import pandas as pd
import random

import warnings
warnings.filterwarnings('ignore')

In [2]:
file_name = 'F01/Session1/wav_arrayMic/0001.wav'
y, sr = librosa.load(file_name, duration=5.0)
ps = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=128)
ps.shape

(128, 216)

In [3]:
import os
import json
import numpy as np
from tqdm import tqdm

# Dataset
D_mfcc = []
data = {
        "mfcc": [],
        "label": []
       }

directories =[  
                 ['f01','Atypical'],['f02','Atypical'],['f03','Atypical'],['f04','Atypical'],['f05','Atypical'],
                 ['f06','Atypical'],['f07','Atypical'],['f08','Atypical'],['f09','Atypical'],['f10','Aypical'],
                 ['f11','Atypical'],
    
                 ['fc01','Typical'],['fc02','Typical'],['fc03','Typical'],['fc04','Typical'],['fc05','Typical'],
                 ['fc06','Typical'],['fc07','Typical'],['fc08','Typical'],['fc09','Typical'],['fc10','Typical'],
    
                 ['m01','Atypical'],['m02','Atypical'],['m03','Atypical'],['m04','Atypical'],['m05','Atypical'],
                 ['m06','Atypical'],['m07','Atypical'],['m08','Atypical'],['m09','Atypical'],['m10','Atypical'],
                 ['m11','Atypical'],['m12','Atypical'],['m13','Atypical'],['m14','Atypical'],['m15','Atypical'],
                 ['m16','Atypical'],['m17','Atypical'],['m18','Atypical'],['m19','Atypical'],['m20','Atypical'],
    
                 ['mc01','Typical'],['mc02','Typical'],['mc03','Typical'],['mc04','Typical'],['mc05','Typical'],
                 ['mc06','Typical'],['mc07','Typical'],['mc08','Typical'],['mc09','Typical'],['mc10','Typical'],
                 ['mc11','Typical'],['mc12','Typical'],['mc13','Typical'],['mc14','Typical']
             ]

for pair in directories:
    directory = pair[0]
    label = pair[1]
    print(directory)
    for filename in os.listdir(directory):
        if filename != 'Notes':
            file = os.path.join(directory, filename)
            file = os.path.join(file, 'wav_arrayMic')
            if os.path.isdir(file):
                for audiofile in tqdm(os.listdir(file)):
                    f = os.path.join(file, audiofile)
                    if os.path.isfile(f):
                        if librosa.get_duration(filename=f) != 0:
                            y, sr = librosa.load(f, duration=5.0)
                            mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=128)
                            if mfcc.shape == (128, 216): 
                                D_mfcc.append( (mfcc, label) )
                                data["mfcc"].append(mfcc.tolist())
                                data["label"].append(label)

  1%|          | 1/134 [00:00<00:14,  9.37it/s]

F01


  0%|          | 1/204 [00:00<00:20,  9.97it/s]

F03


  1%|          | 1/199 [00:00<00:19,  9.98it/s]

...

  0%|          | 1/600 [00:00<01:09,  8.63it/s]

MC03


  0%|          | 1/648 [00:00<01:10,  9.24it/s]

MC04


100%|██████████| 373/373 [00:16<00:00, 22.25it/s]


In [4]:
with open('MFCC_DATA.json', "w") as fp:
        json.dump(data, fp, indent=4)

In [7]:
with open('MFCC_DATA.json', "r") as f:
    data = json.load(f)

X = np.array(data["mfcc"])
y = np.array(data["label"])
D = np.array(data)


In [13]:
X = np.array((X-np.min(X))/(np.max(X)-np.min(X)))
X = X/np.std(X)
y = np.array(y)

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize

#Split twice to get the validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.053, random_state=123, stratify=y)
X_train, X_validate, y_train, y_validate = train_test_split(X_train, y_train, test_size=0.157, random_state=123)
#Print the shapes
X_train.shape, X_test.shape, X_validate.shape, len(y_train), len(y_test), len(y_validate)

((800, 128, 216), (54, 128, 216), (150, 128, 216), 800, 54, 150)

In [36]:
Y = y_test

In [37]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
#y=to_categorical(labelencoder.fit_transform(y))
#y
y_train = np.array(to_categorical(labelencoder.fit_transform(y_train)))
y_validate = np.array(to_categorical(labelencoder.fit_transform(y_validate)))
y_test = np.array(to_categorical(labelencoder.fit_transform(y_test)))

In [38]:
X_train.shape, X_validate.shape, X_test.shape, y_train.shape, y_validate.shape, y_test.shape

((800, 128, 216), (150, 128, 216), (54, 128, 216), (800, 2), (150, 2), (54, 2))

In [39]:
import tensorflow.keras as keras
from keras.layers import Activation, Dense, Dropout, Conv2D, \
                         Flatten, MaxPooling2D
from tensorflow.keras.layers import LSTM, Dense

input_shape = (X_train.shape[1], X_train.shape[2])
model = keras.Sequential()
model.add(LSTM(128,input_shape=input_shape))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(48, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               176640    
_________________________________________________________________
module_wrapper_3 (ModuleWrap (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_5 (Dense)              (None, 64)                8256      
_________________________________________________________________
module_wrapper_4 (ModuleWrap (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 48)                3120      
_________________________________________________________________
module_wrapper_5 (ModuleWrap (None, 48)               

In [40]:
model.compile(
	optimizer="Adam",
	loss="categorical_crossentropy",
	metrics=['accuracy'])

model.fit(
	x=X_train, 
	y=y_train,
    epochs=50,
    batch_size=32,
    validation_data= (X_validate, y_validate))

score = model.evaluate(
	x=X_test,
	y=y_test)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/50
25/25 [==============================] - 5s 99ms/step - loss: 0.7120 - accuracy: 0.5013 - val_loss: 0.7146 - val_accuracy: 0.4067
Epoch 2/50
25/25 [==============================] - 2s 80ms/step - loss: 0.7110 - accuracy: 0.4638 - val_loss: 0.7035 - val_accuracy: 0.4067
Epoch 3/50
25/25 [==============================] - 2s 78ms/step - loss: 0.6939 - accuracy: 0.5200 - val_loss: 0.7104 - val_accuracy: 0.4067
Epoch 4/50
25/25 [==============================] - 2s 76ms/step - loss: 0.6982 - accuracy: 0.4988 - val_loss: 0.6915 - val_accuracy: 0.5933
Epoch 5/50
25/25 [==============================] - ETA: 0s - loss: 0.6984 - accuracy: 0.47 - 2s 76ms/step - loss: 0.6984 - accuracy: 0.4762 - val_loss: 0.6975 - val_accuracy: 0.4067
Epoch 6/50
25/25 [==============================] - 2s 76ms/step - loss: 0.6941 - accuracy: 0.4963 - val_loss: 0.7010 - val_accuracy: 0.4067
Epoch 7/50
25/25 [==============================] - 2s 81ms/step - loss: 0.6930 - accuracy: 0.5113 - val_loss: 0

In [41]:
predict = model.predict_classes(X_test)
prediction_class = labelencoder.inverse_transform(predict)
#real_class = labelencoder.inverse_transform(y_test)
#prediction_class

y = Y
match = 0
print('  Real       Prediction       Match')
for i in range(0,X_test.shape[0]):
    if(y[i]==prediction_class[i]):
        match+=1
    print(y[i] + '       ' + prediction_class[i] + '       ' + str(y[i]==prediction_class[i]))
print('Matched = ', match)
print('Not Matched = ',(X_test.shape[0] - match))

  Real       Prediction       Match
Typical       Typical       True
Typical       Typical       True
Typical       Typical       True
Atypical       Typical       False
Typical       Typical       True
Atypical       Typical       False
Atypical       Typical       False
Atypical       Typical       False
Typical       Typical       True
Atypical       Typical       False
Typical       Typical       True
Typical       Typical       True
Atypical       Typical       False
Atypical       Typical       False
Atypical       Typical       False
Typical       Typical       True
Atypical       Typical       False
Typical       Typical       True
Typical       Typical       True
Atypical       Typical       False
Typical       Typical       True
Atypical       Typical       False
Typical       Typical       True
Atypical       Typical       False
Typical       Typical       True
Typical       Typical       True
Typical       Typical       True
Atypical       Typical       False
Atypical      